In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # seaborn for plotting
import matplotlib.pyplot as plt # graphing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#
# load data and store the target separately, then drop from training
#

train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv', index_col='id', header=0)
y = train_df['target']
train_df = train_df.drop(['target'],axis=1)


In [ ]:
#
# do some EDA
#

#print(train_df.head())
print(train_df.describe())

train_corr = train_df.corr()
df_lt = train_corr.where(np.tril(np.ones(train_corr.shape)).astype(np.bool))
sns.heatmap(df_lt,cmap="rocket",
            xticklabels=train_corr.columns,
            yticklabels=train_corr.columns
           )


In [ ]:
#
# feature engineering
#
# cont6 looks like it should be taken out, as it has high correlation with several other variables
# remove it and check again

col_remove_list = ['cont6']

train_trim_df = train_df.drop(col_remove_list, axis=1)
train_corr = train_trim_df.corr()
df_lt = train_corr.where(np.tril(np.ones(train_corr.shape)).astype(np.bool))
sns.heatmap(df_lt,cmap="rocket",
            xticklabels=train_corr.columns,
            yticklabels=train_corr.columns
           )

In [ ]:
# OK so let's see what happens when we try a simple linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

random_seed = 1234
X_train, X_test, y_train, y_test = train_test_split(train_trim_df, y, random_state=random_seed)
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
score = mean_squared_error(y_pred, y_test)
print(np.sqrt(score))

In [ ]:
#
# create our base model for submission
#
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv', index_col='id',header=0)
test_df = test_df.drop(col_remove_list, axis=1)

y_test_pred = lr.predict(test_df)

output = pd.DataFrame({"id":test_df.index,'target':y_test_pred})
output.to_csv('./submission.csv',index=False)